# Code for practical part of PDC Project

Hard limits to respect :
- Codewords have to be in the space [-3/2, 3/2] x [-3/2, 3/2] -> **limits the energy per symbol**
- The average energy of the vector **X** should be <= 1 -> **also limits the energy per symbol** 
- The dimension of the encoded vector **X** should not exceed n <= 100 -> **limits the number of symbols we can send**

In [100]:
### IS THE ENERGY PER SYMBOL A HYPER-PARAMETER ???
import numpy as np
import scipy as sp

## Utility functions to help code the encoder/decoder

In [101]:
# Computes the value of the Q-Function
def q_function(x):
    return (1/2) * sp.erfc(x/np.sqrt(2))

# Gets every possible codeword lengths for which the length divides the total length of the original bitstring
def compute_codeword_lengths(bit_str):
    n = len(bit_str)
    return [i for i in range(1, n+1) if n % i == 0]

# Gets the whole alphabet from a codeword length
def get_alphabet_from_codeword_length(length):
    return [f'{i:0{length}b}' for i in range(0, 2**length)]

# Performs a conversion from ASCII string to binary symbols
def ascii_str_to_binary(ascii_str):
    return str(''.join(format(ord(i), '08b') for i in ascii_str))

def split_bit_str(bit_str, chunk_size):
    return [bit_str[i:i+chunk_size] for i in range(0, len(bit_str), chunk_size)]


# # Gets the encoded alphabet
# def encode_alphabet(energy_per_symbol, alphabet, encoder):
#     return [encoder(energy_per_symbol, i, alphabet) for i in alphabet]


## Functions for the types of encodings

### m-PSK

In [102]:
# Computes the probability of error for symbol i with PSK encoding (P_e(i) == P_e here !)
def error_m_psk(i, m, energy_per_symbol, noise_var = 10**(-2.65)):
    inner_part = lambda theta: np.exp(-(np.sin(np.pi/m)**2)**2 / np.sin(theta) * (energy_per_symbol/(2*(noise_var**2))))
    return (1/np.pi) * sp.quad(inner_part, 0, np.pi - (np.pi / m))

# Performs the m-PSK encoding
def m_psk_encoder(energy_per_symbol, codeword, alphabet):
    k = alphabet.index(codeword)
    m = len(alphabet)
    return np.sqrt(energy_per_symbol) * np.exp(2j * np.pi * (k/m))

### QAM

In [103]:
# Computes the probability of error for symbol i with QAM encoding (P_e(i) == P_e here !)
def error_qam(i, d, m, energy_per_symbol, noise_var = 10**(-2.65)):
    func = q_function(d/(2*noise_var))
    return 2*func - func**2

def qam_encoder(energy_per_symbol, codeword, alphabet):
    return

## Encoder part

In [104]:
def encode_string(raw_str, chunk_size, energy_per_symbol, theta_estimator_batch_size, encoder):
    if theta_estimator_batch_size >= 100:
        raise OverflowError("ERROR: The batch of dummy samples cannot be equal or exceed 100 symbols.") 
    bit_str = ascii_str_to_binary(raw_str)
    splitted_bit_str = split_bit_str(bit_str, chunk_size)
    splitted_bit_str_size = len(splitted_bit_str)
    if splitted_bit_str_size > 100:
        raise OverflowError("ERROR: The string without the dummy symbols cannot exceed 100 symbols.") 

    if splitted_bit_str_size + theta_estimator_batch_size > 100:
        theta_estimator_batch_size = 100 - splitted_bit_str_size

    alphabet = get_alphabet_from_codeword_length(chunk_size)
    theta_estimator_batch = np.full((theta_estimator_batch_size, 1), alphabet[0])
    full_str = np.append(theta_estimator_batch, splitted_bit_str)
    return np.array([encoder(energy_per_symbol, codeword, alphabet) for codeword in full_str])

## Channel part (given)

In [105]:
def channel(sent_signal):
    s = np.mean(sent_signal**2)
    if s <= 1:
        s = 1
    noise_power = (10**(-2.65))*s
    shift = np.exp(-2j*np.pi*np.random.rand())
    sent_signal = sent_signal*shift
    noise_std = np.sqrt(noise_power/2)
    rcv_signal = sent_signal + noise_std*np.random.randn(len(sent_signal)) + 1j*noise_std*np.random.randn(len(sent_signal))
    return rcv_signal

## Decoder part

In [106]:
def decode_str(noisy_str, decoder, n_dummy_symbols):
    dummy_symbols = noisy_str[0:n_dummy_symbols]
    phase = np.mean(dummy_symbols) # TODO: CHECK HOW TO ISOLATE THE ANGLE FROM THE EXPONENTIAL IN THE EQUATION WITH THE MEAN
    return

## Putting everything together

In [120]:
try:
    encoded_str = encode_string("HELLO", 4, 1, 10, m_psk_encoder)
    print(encoded_str)
    print()
    noisy_result = channel(encoded_str)
    print(noisy_result)
    # decoded_str = decode_str(decoder_str)
    # print(decoded_str)
except OverflowError as ovferr :
    print(ovferr)
    raise

[ 1.00000000e+00+0.00000000e+00j  1.00000000e+00+0.00000000e+00j
  1.00000000e+00+0.00000000e+00j  1.00000000e+00+0.00000000e+00j
  1.00000000e+00+0.00000000e+00j  1.00000000e+00+0.00000000e+00j
  1.00000000e+00+0.00000000e+00j  1.00000000e+00+0.00000000e+00j
  1.00000000e+00+0.00000000e+00j  1.00000000e+00+0.00000000e+00j
  6.12323400e-17+1.00000000e+00j -1.00000000e+00+1.22464680e-16j
  6.12323400e-17+1.00000000e+00j -3.82683432e-01+9.23879533e-01j
  6.12323400e-17+1.00000000e+00j -1.83697020e-16-1.00000000e+00j
  6.12323400e-17+1.00000000e+00j -1.83697020e-16-1.00000000e+00j
  6.12323400e-17+1.00000000e+00j  9.23879533e-01-3.82683432e-01j]

[-0.9813346 -0.16777551j -0.98938468-0.10853612j -0.97027074-0.1301299j
 -0.93415095-0.15800788j -0.95166489-0.10176881j -1.00556621-0.09023228j
 -0.94787532-0.158045j   -1.01226152-0.14397991j -0.9660815 -0.18536501j
 -1.00734409-0.15639287j  0.07020618-1.05168376j  0.98647353+0.16796031j
  0.19033574-0.96229249j  0.5371612 -0.84494657j  0.19694